# gensimを用いた単語埋め込みの学習

単語埋め込みは、単語の表現方法の1つです。このノートブックでは、genismを使って埋め込みを学習する方法を紹介します。[gensim](https://radimrehurek.com/gensim/index.html)は、第7章で解説するトピックモデルなどを含む、自然言語処理用のPythonライブラリです。

## パッケージのインポート

In [7]:
import time
import warnings

from gensim.models import FastText, Word2Vec, KeyedVectors   
from gensim.corpora.wikicorpus import WikiCorpus
warnings.filterwarnings('ignore')

## 学習データの準備

In [8]:
# gensimのword2vecでは、リストのリスト形式での学習データが必要です。
# 各リストが1つの文書を表し、文書はトークンのリストで表されます。
corpus = [
    ['dog', 'bites', 'man'],
    ['man', 'bites', 'dog'],
    ['dog', 'eats', 'meat'],
    ['man', 'eats', 'food']
]

## Continuous Bag of Words (CBOW) 

CBOWの主なタスクは、文脈語を与えたときに、その中心語を正しく予測できる言語モデルを構築することです。
gensimのWord2Vecクラスに学習データとハイパーパラメータを指定することで学習できます。
学習アルゴリズムは、`sg`パラメータで指定します。CBOWを使うなら`0`、Skip-gramを使うなら`1`を指定します。その他のパラメータについては、以下のドキュメントを参照してください。

- [gensim.models.word2vec.Word2Vec](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec)

In [9]:
# モデルの学習
model_cbow = Word2Vec(corpus, min_count=1, sg=0)

# モデルのサマリー
print(model_cbow)

# ボキャブラリの表示
words = list(model_cbow.wv.vocab)
print(words)

# 単語ベクトルの取得
print(model_cbow.wv['dog'])

Word2Vec(vocab=6, size=100, alpha=0.025)
['dog', 'bites', 'man', 'eats', 'meat', 'food']
[-2.4190836e-03  3.9345301e-03 -4.0867063e-03 -3.3580605e-03
 -4.9987063e-03 -3.2243780e-03 -1.9584254e-03 -3.2025778e-03
  3.9975140e-03 -3.8498973e-03  4.6813912e-03 -3.1696623e-03
 -5.1643112e-04  4.3939748e-03 -2.6446586e-03  4.2424193e-03
  9.3994418e-04 -3.5626194e-03 -1.6576669e-03  1.1161768e-04
 -4.9683126e-04 -3.5317065e-03 -2.0356272e-05  4.3223472e-03
 -4.8883087e-03  8.2331704e-04  1.1718493e-03  2.3582387e-03
 -4.2451480e-03  2.2809638e-03  5.7795114e-04 -4.5479289e-03
  3.8631337e-03  2.1488585e-03 -1.8883956e-03 -1.0207531e-03
 -2.6229296e-03 -4.1782241e-03 -7.3080783e-04 -2.2069707e-03
 -2.5679416e-03 -2.4002710e-04  3.4047170e-03 -3.0350059e-03
  1.2890766e-03 -3.9590728e-03 -1.9741219e-03 -1.5056815e-03
  2.0105110e-03 -2.0129199e-03 -3.0427284e-03  4.3714312e-03
 -4.5723256e-04 -2.9080869e-03 -3.7506886e-03  2.0689457e-03
  4.8298906e-03  1.7303320e-03  6.1707827e-04 -1.7938673e

[similarity](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.similarity)メソッドを使って、単語の類似度を計算します。similarityメソッドに2つの単語を与えて呼び出すことで、学習した単語ベクトル同士のコサイン類似度を計算しています。

In [10]:
# 類似度の計算
print("Similarity between eats and bites:", model_cbow.wv.similarity('eats', 'bites'))
print("Similarity between eats and man:", model_cbow.wv.similarity('eats', 'man'))

Similarity between eats and bites: -0.03992596
Similarity between eats and man: -0.046774115


上記の類似度スコアを見ると、eatsはmanよりbitesに近いことがわかります。

次に、[most_similar](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.most_similar)メソッドを使って、与えた単語に最も類似した単語を検索してみましょう。このメソッドに単語を与えて呼び出すことで、ボキャブラリ内のその他の単語とのコサイン類似度を計算して、類似度順に返してくれます。その他の機能として、`King - Man + Woman = Queen`のような類推問題を解くこともできます。詳細については、[公式ドキュメント](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.most_similar)を参照してください。

In [11]:
model_cbow.wv.most_similar('meat')

[('man', 0.15192528069019318),
 ('food', 0.12299454212188721),
 ('eats', 0.1195271834731102),
 ('bites', -0.014911144971847534),
 ('dog', -0.16333791613578796)]

In [12]:
# モデルの保存
model_cbow.save('model_cbow.bin')

# モデルの読み込み
new_model_cbow = Word2Vec.load('model_cbow.bin')
print(new_model_cbow)

Word2Vec(vocab=6, size=100, alpha=0.025)


## SkipGram

Skipgramのタスクは、中心語から文脈語を予測することです。

In [13]:
# モデルの学習
model_skipgram = Word2Vec(corpus, min_count=1, sg=1)

# モデルのサマリー
print(model_skipgram)

# ボキャブラリの表示
words = list(model_skipgram.wv.vocab)
print(words)

# 単語ベクトルの取得
print(model_skipgram.wv['dog'])

Word2Vec(vocab=6, size=100, alpha=0.025)
['dog', 'bites', 'man', 'eats', 'meat', 'food']
[-2.4190836e-03  3.9345301e-03 -4.0867063e-03 -3.3580605e-03
 -4.9987063e-03 -3.2243780e-03 -1.9584254e-03 -3.2025778e-03
  3.9975140e-03 -3.8498973e-03  4.6813912e-03 -3.1696623e-03
 -5.1643112e-04  4.3939748e-03 -2.6446586e-03  4.2424193e-03
  9.3994418e-04 -3.5626194e-03 -1.6576669e-03  1.1161768e-04
 -4.9683126e-04 -3.5317065e-03 -2.0356272e-05  4.3223472e-03
 -4.8883087e-03  8.2331704e-04  1.1718493e-03  2.3582387e-03
 -4.2451480e-03  2.2809638e-03  5.7795114e-04 -4.5479289e-03
  3.8631337e-03  2.1488585e-03 -1.8883956e-03 -1.0207531e-03
 -2.6229296e-03 -4.1782241e-03 -7.3080783e-04 -2.2069707e-03
 -2.5679416e-03 -2.4002710e-04  3.4047170e-03 -3.0350059e-03
  1.2890766e-03 -3.9590728e-03 -1.9741219e-03 -1.5056815e-03
  2.0105110e-03 -2.0129199e-03 -3.0427284e-03  4.3714312e-03
 -4.5723256e-04 -2.9080869e-03 -3.7506886e-03  2.0689457e-03
  4.8298906e-03  1.7303320e-03  6.1707827e-04 -1.7938673e

In [14]:
print("Similarity between eats and bites:", model_skipgram.wv.similarity('eats', 'bites'))
print("Similarity between eats and man:", model_skipgram.wv.similarity('eats', 'man'))

Similarity between eats and bites: -0.039925296
Similarity between eats and man: -0.046784468


In [15]:
model_skipgram.wv.most_similar('meat')

[('man', 0.15192528069019318),
 ('food', 0.1229945495724678),
 ('eats', 0.11946240067481995),
 ('bites', -0.014911137521266937),
 ('dog', -0.16333791613578796)]

In [16]:
model_skipgram.save('model_skipgram.bin')

new_model_skipgram = Word2Vec.load('model_skipgram.bin')
print(model_skipgram)

Word2Vec(vocab=6, size=100, alpha=0.025)


## Wikiコーパスを用いた埋め込みの学習

ここまでで、小さなデータを使ってモデルを学習する方法は紹介しました。
次は、より大きなデータを使ってモデルを学習してみましょう。
ただ、そのすべてを利用すると全体で数GBを超えるので、今回はその一部だけを使って、word2vecとfastTextの埋め込みを学習します。コーパスは以下のページからダウンロードできます。

- https://dumps.wikimedia.org/jawiki/20210701/


### データのダウンロード

In [17]:
!mkdir -p data/ja/
!wget -P data/ja/ https://dumps.wikimedia.org/jawiki/20210701/jawiki-20210701-pages-articles6.xml-p4307948p4407107.bz2

--2021-09-02 00:07:27--  https://dumps.wikimedia.org/jawiki/20210701/jawiki-20210701-pages-articles6.xml-p4307948p4407107.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.7, 2620:0:861:1:208:80:154:7
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 52534161 (50M) [application/octet-stream]
Saving to: ‘data/ja/jawiki-20210701-pages-articles6.xml-p4307948p4407107.bz2’

jawiki-20210701-pag 100%[===================>]  50.10M  4.76MB/s    in 10s     

2021-09-02 00:07:38 (4.80 MB/s) - ‘data/ja/jawiki-20210701-pages-articles6.xml-p4307948p4407107.bz2’ saved [52534161/52534161]



### パッケージのインストール

日本語の場合、学習前に単語分割が必要なので、形態素解析器「MeCab」のラッパーである「fugashi」をインストールします。

In [18]:
!pip install fugashi[unidic-lite]

     |████████████████████████████████| 490 kB 5.0 MB/s 
     |████████████████████████████████| 47.4 MB 43 kB/s 
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658836 sha256=6c8473b47e878c2446fff6a8dee749033d9bbbd52987347eebc0707132c470bb
  Stored in directory: /root/.cache/pip/wheels/de/69/b1/112140b599f2b13f609d485a99e357ba68df194d2079c5b1a2
Successfully built unidic-lite


### 前処理

インストールを終えたら、単語分割用の関数を定義します。
学習データの準備にgensimのWikiCorpusを利用するため、その内部で必要とされる形式の関数を定義しています。パラメータは以下の通りです。

- content (str): Wikiのマークアップを除去した文字列
- token_min_len (int): 最小のトークン長
- token_max_len (int): 最大のトークン長
- lower (bool): 文字列を小文字化するか否か

その他の詳細については、[公式ドキュメント](https://radimrehurek.com/gensim/corpora/wikicorpus.html#gensim.corpora.wikicorpus.tokenize)を参照してください。

In [19]:
from fugashi import Tagger
from gensim.corpora.wikicorpus import TOKEN_MAX_LEN, WikiCorpus
from gensim import utils
fugger = Tagger('-Owakati')
TOKEN_MIN_LEN = 1

def tokenize(
    content,
    token_min_len=TOKEN_MIN_LEN,
    token_max_len=TOKEN_MAX_LEN,
    lower=True
  ):
    return [
        utils.to_unicode(token.surface) for token in fugger(content.lower() if lower else content)
    ]

単語分割用の関数を定義したら、学習データを準備します。
学習データの準備にはgensimの[WikiCorpus](https://radimrehurek.com/gensim/corpora/wikicorpus.html#gensim.corpora.wikicorpus.WikiCorpus)クラスを使います。
WikiCorpusクラスを使うことで、Wikipediaの記事のダンプを、学習データとして扱うことができます。サポートしているダンプのフォーマットは以下の2種類です。

- \<LANG\>wiki-\<YYYYMMDD\>-pages-articles.xml.bz2
- \<LANG\>wiki-latest-pages-articles.xml.bz2

In [20]:
# 学習データの準備
wiki = WikiCorpus(
    'data/ja/jawiki-20210701-pages-articles6.xml-p4307948p4407107.bz2',
    tokenizer_func=tokenize
)
sentences = list(wiki.get_texts())

### Word2Vecモデルの学習

In [21]:
# CBOWの学習
start = time.time()
word2vec_cbow = Word2Vec(sentences, min_count=10, sg=0)
end = time.time()

print("Time taken for training is:{:.2f} hrs ".format((end - start) / 3600.0))

Time taken for training is:0.02 hrs 


In [22]:
print(word2vec_cbow)
print("-" * 30)

# ボキャブラリの取得
words = list(word2vec_cbow.wv.vocab)
print(words[:10])
print("-" * 30)

# ベクトルの取得
print(word2vec_cbow.wv['猫'])
print("-" * 30)

# 類似度の計算
print("Similarity between 猫 and 犬:", word2vec_cbow.wv.similarity('猫', '犬'))
print("Similarity between 猫 and 石:", word2vec_cbow.wv.similarity('猫', '石'))
print("-" * 30)

Word2Vec(vocab=55023, size=100, alpha=0.025)
------------------------------
["'''", '松田', "'''（", 'まつだ', 'みのり', '、', '1886', '年', '（', '明治']
------------------------------
[ 0.4803291  -0.26755214  1.8851796   2.1370397   0.4145621   2.4362938
  1.091774   -0.79165006  1.0529066   1.1643319  -0.6166124  -1.3831989
  0.32871732  1.7413331   0.45214954  0.3553559  -0.51122105 -0.1195362
  1.0193094  -1.4569782   2.2853062  -0.14232646 -0.8975287  -2.184406
 -0.4996832   0.68254656 -0.38484892  1.1567738  -0.1356831   0.43790963
  0.02898838 -0.22504367 -0.20682266 -1.6368281   1.5475001  -1.7462156
 -1.1966965   0.4772035  -1.6926249   0.59005827  1.2758892  -1.6047785
  0.40053117 -0.10537413  0.8446828   1.3618791  -0.00406642  0.6349208
 -2.5736046  -1.789437    0.05310813 -0.78653073 -1.2193427  -0.5214791
 -1.6702218  -0.6017423  -0.22100064  0.2051663   0.6978673  -0.6835604
  0.5850414   0.41687828  0.836443    1.3891189  -1.0133115   1.4724859
 -0.48280108 -0.7773937   0.2819049 

In [23]:
# モデルの保存
word2vec_cbow.wv.save_word2vec_format('word2vec_cbow.bin', binary=True)

# モデルの読み込み
# new_modelword2vec_cbow = Word2Vec.load('word2vec_cbow.bin')
# print(word2vec_cbow)

In [24]:
# SkipGramの学習
start = time.time()
word2vec_skipgram = Word2Vec(sentences, min_count=10, sg=1)
end = time.time()

print("Time taken for training is:{:.2f} hrs ".format((end - start) / 3600.0))

Time taken for training is:0.07 hrs 


In [25]:
print(word2vec_skipgram)
print("-" * 30)

words = list(word2vec_skipgram.wv.vocab)
print(words[:10])
print("-" * 30)

print(word2vec_skipgram.wv['猫'])
print("-" * 30)

print("Similarity between 猫 and 犬:", word2vec_skipgram.wv.similarity('猫', '犬'))
print("Similarity between 猫 and 石:", word2vec_skipgram.wv.similarity('猫', '石'))
print("-" * 30)

Word2Vec(vocab=55023, size=100, alpha=0.025)
------------------------------
["'''", '松田', "'''（", 'まつだ', 'みのり', '、', '1886', '年', '（', '明治']
------------------------------
[ 3.18760425e-01  3.19911897e-01 -1.67814702e-01  7.50234246e-01
  2.66804278e-01  1.92610651e-01  1.44445568e-01 -1.40261889e-01
 -7.53758773e-02  4.64176238e-02 -4.21440974e-02 -5.36385715e-01
  1.89105153e-01 -2.10110098e-01 -8.47956419e-01  7.54157454e-02
  2.87250191e-01  5.83331585e-01  3.39055866e-01  9.46621224e-02
 -9.03506801e-02  3.40238392e-01 -5.59616089e-03 -2.30783120e-01
 -2.51028419e-01  1.15221448e-01  3.05352092e-01 -1.13746695e-01
 -9.35844705e-02  4.86865565e-02  3.87228727e-01 -1.98963463e-01
 -6.63830787e-02 -5.11656821e-01  4.69573170e-01 -3.19391191e-01
 -3.20576906e-01  2.73486687e-04 -4.02929902e-01  3.82524654e-02
  5.70152521e-01 -1.09046154e-01  4.42002714e-02 -4.49504107e-02
  5.82416356e-02  5.19746423e-01  3.53730947e-01  2.01171011e-01
 -5.94525516e-01 -4.88940418e-01  1.10941686e-01

In [26]:
# モデルの保存
word2vec_cbow.wv.save_word2vec_format('word2vec_sg.bin', binary=True)

# モデルの読み込み
# new_model_skipgram = Word2Vec.load('model_skipgram.bin')
# print(model_skipgram)

### FastTextモデルの学習

In [27]:
# CBOWの学習
start = time.time()
fasttext_cbow = FastText(sentences, sg=0, min_count=10)
end = time.time()

print("Time taken for training is:{:.2f} hrs ".format((end - start) / 3600.0))

Time taken for training is:0.04 hrs 


In [28]:
print(fasttext_cbow)
print("-" * 30)

words = list(fasttext_cbow.wv.vocab)
print(words[:10])
print("-" * 30)

print(fasttext_cbow.wv['猫'])
print("-" * 30)

print("Similarity between 猫 and 犬:", fasttext_cbow.similarity('猫', '犬'))
print("Similarity between 猫 and 石:", fasttext_cbow.similarity('猫', '石'))
print("-" * 30)

FastText(vocab=55023, size=100, alpha=0.025)
------------------------------
["'''", '松田', "'''（", 'まつだ', 'みのり', '、', '1886', '年', '（', '明治']
------------------------------
[-0.21055603  1.9364047  -0.6618966  -2.8286877  -1.747594    0.8876541
  1.9856669   1.2447248  -0.82500076  0.42230105 -0.04942975 -0.63433045
  2.763354    0.2237634  -0.75201464  0.24883053 -0.54105586 -0.64725536
 -2.368435    0.13232313 -0.4485092  -0.27970585  1.5105442   0.47174805
  1.3213906   0.09933054  0.20629425  0.48365623  0.04136713 -0.34680226
  1.1115282  -0.5453791   0.27316573 -1.6178588   0.86495936  0.52859753
 -0.01277385 -1.8511162  -1.4706675  -1.2783549   0.6667346  -0.974852
  0.10005664 -0.48317418 -0.42455682  2.4559274   1.0890949   0.2856696
  1.5529332  -1.8901747  -0.4491058   0.8683636   1.9854089   0.26533592
  0.60221875 -0.7380746   0.6612123   0.61387116  0.26555738 -1.3728558
 -0.3879201  -0.45068723 -1.1440125  -2.2049565   1.430046   -1.2279831
  1.358506    0.23567013 -0.495

In [29]:
# SkipGramの学習
start = time.time()
fasttext_skipgram = FastText(sentences, sg=1, min_count=10)
end = time.time()

print("Time taken for training is:{:.2f} hrs ".format((end - start) / 3600.0))

Time taken for training is:0.09 hrs 


In [30]:
print(fasttext_skipgram)
print("-" * 30)

words = list(fasttext_skipgram.wv.vocab)
print(words[:10])
print("-" * 30)

print(fasttext_skipgram.wv['猫'])
print("-" * 30)

print("Similarity between 猫 and 犬:", fasttext_skipgram.wv.similarity('猫', '犬'))
print("Similarity between 猫 and 石:", fasttext_skipgram.wv.similarity('猫', '石'))
print("-" * 30)

FastText(vocab=55023, size=100, alpha=0.025)
------------------------------
["'''", '松田', "'''（", 'まつだ', 'みのり', '、', '1886', '年', '（', '明治']
------------------------------
[ 0.2696672   0.18861265 -0.02024899  0.00622671 -0.00185222  0.14164612
  0.2409192  -0.21536526  0.10057747 -0.13081904 -0.12770903 -0.4597932
  0.20203054  0.0411027  -0.25389844 -0.23035528  0.08346335  0.03206843
 -0.42905587  0.25714007 -0.27228212 -0.04105185 -0.44270617 -0.2758658
  0.05847213  0.1434209  -0.02533826 -0.16879071 -0.05606332  0.24250448
  0.25177985  0.13241014 -0.21466193  0.14706865 -0.3531927   0.271098
  0.32020685  0.04217644 -0.1842621  -0.06005319 -0.19842038  0.26519722
 -0.42355868  0.21075994 -0.26790833  0.11443414 -0.01542821  0.06442037
 -0.00555118  0.02112193  0.11594013  0.12190726  0.2428713  -0.04728377
 -0.02653726  0.07947062 -0.03165951  0.40582758 -0.01276721  0.21176325
 -0.10584069 -0.06230808  0.10854024 -0.11993311 -0.29278514 -0.36715135
  0.03995505  0.2526267   0.1